In [2]:
# Importing necessary libraries.
import pandas as pd
import numpy as np

# import TfidfVector from sklearn.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
csv_file_path = r'E:\Collage\Semester-4\People Analytics\Project part 1\Code2\fau_airlines_onboarding.csv'
employees = pd.read_csv(csv_file_path)
employees

,id,study degree,previous_experience,civil_status,personality_traits,hobbies,favourite sport
0,emp_001,economics,Expert.,married,"Extrovert, enjoys group work and likes to comm...","Interest centered around learning, reading and...",tennis
1,emp_002,business,Novice.,single,"Intelligent, inquisitive and seeks out knowledge","Interest centered around fitness, health, and ...",volleyball
2,emp_003,media,Advanced beginner.,married,"Introvert, empathetic, prefers a close circle ...",Keen interest in finance and investment,volleyball
3,emp_004,psychology,Competent.,single,"Helpful, emotionally intelligent and gets alon...",Interest centered around physical activities a...,tennis
4,emp_005,economics,Proficient.,single,"Extrovert, enjoys group work and likes to comm...","Interest centered around fitness, health, and ...",swimming
...,...,...,...,...,...,...,...
95,emp_096,media,Advanced beginner.,in a relationship,"Helpful, emotionally intelligent and gets alon...","Interest centered around fitness, health, and ...",tennis
96,emp_097,communication,Novice.,in a relationship,"Extrovert, expresses themselves easily, outgoi...","Interest centered around learning, reading and...",volleyball
97,emp_098,communication,Advanced beginner.,single,Easily upset and overly critical of the self,"Interest centered around fitness, health, and ...",swimming
98,emp_099,psychology,Competent.,in a relationship,"Helpful, emotionally intelligent and gets alon...","Interest centered around fitness, health, and ...",swimming


In [4]:
def create_soup(x):
    return ''.join(x['study degree']) + ''.join(x['previous_experience']) + '' + ''.join(x['civil_status']) + '' + ''.join(x['personality_traits']) + '' + ''.join(x['hobbies']) + '' + ''.join(x['favourite sport'])
employees['soup'] = employees.apply(create_soup, axis=1)

In [5]:
# Create TF-IDF vector
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(employees['soup'])
tfidf_matrix.shape

(100, 107)

In [6]:
# Compute similarity score between the employees
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [8]:
# construct a reverse map of indices and employee IDs
indices = pd.Series(employees.index, index=employees['id']).drop_duplicates()

In [9]:
def get_recommendations(ID, cosine_sim=cosine_sim):
    
    # get the index of the employee that matches the employee ID
    IDx = indices[ID]
    
    # get the pairwise similarity scores of all employees with the specified employee ID
    sim_scores = list(enumerate(cosine_sim[IDx]))
    
    # sort employees based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # get the scores of the three most similar employees
    sim_scores = sim_scores[1:4]
    
    # get employee indices
    employees_indices = [i[0] for i in sim_scores]
    
    # return the top three most similar employees
    return employees['id'].iloc[employees_indices]

In [10]:
get_recommendations('emp_100', cosine_sim)

41    emp_042
88    emp_089
75    emp_076
Name: id, dtype: object

In [11]:
new_row = {'id': 'emp_101', 'study degree': 'business', 'previous_experience': 'Competent.', 'civil_status': 'single', 'personality_traits': 'Extrovert, expresses themselves easily, outgoing and seeks out social interaction and conversations', 'hobbies': 'Interest centered around learning, reading and creative art', 'favourite sport': 'football'}
employees = employees._append(new_row, ignore_index=True)


In [14]:
employees['soup'] = employees.apply(create_soup, axis=1)

In [13]:
employees

,id,study degree,previous_experience,civil_status,personality_traits,hobbies,favourite sport,soup
0,emp_001,economics,Expert.,married,"Extrovert, enjoys group work and likes to comm...","Interest centered around learning, reading and...",tennis,"economicsExpert.marriedExtrovert, enjoys group..."
1,emp_002,business,Novice.,single,"Intelligent, inquisitive and seeks out knowledge","Interest centered around fitness, health, and ...",volleyball,"businessNovice.singleIntelligent, inquisitive ..."
2,emp_003,media,Advanced beginner.,married,"Introvert, empathetic, prefers a close circle ...",Keen interest in finance and investment,volleyball,"mediaAdvanced beginner.marriedIntrovert, empat..."
3,emp_004,psychology,Competent.,single,"Helpful, emotionally intelligent and gets alon...",Interest centered around physical activities a...,tennis,"psychologyCompetent.singleHelpful, emotionally..."
4,emp_005,economics,Proficient.,single,"Extrovert, enjoys group work and likes to comm...","Interest centered around fitness, health, and ...",swimming,"economicsProficient.singleExtrovert, enjoys gr..."
...,...,...,...,...,...,...,...,...
96,emp_097,communication,Novice.,in a relationship,"Extrovert, expresses themselves easily, outgoi...","Interest centered around learning, reading and...",volleyball,communicationNovice.in a relationshipExtrovert...
97,emp_098,communication,Advanced beginner.,single,Easily upset and overly critical of the self,"Interest centered around fitness, health, and ...",swimming,communicationAdvanced beginner.singleEasily up...
98,emp_099,psychology,Competent.,in a relationship,"Helpful, emotionally intelligent and gets alon...","Interest centered around fitness, health, and ...",swimming,"psychologyCompetent.in a relationshipHelpful, ..."
99,emp_100,psychology,Advanced beginner.,in a relationship,"Extrovert, expresses themselves easily, outgoi...","Interest in around reading and creative art, f...",swimming,psychologyAdvanced beginner.in a relationshipE...


In [15]:
# Create TF-IDF vector

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(employees['soup'])
tfidf_matrix.shape

(101, 108)

In [17]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [18]:
# construct a reverse map of indices and employee IDs
indices = pd.Series(employees.index, index=employees['id']).drop_duplicates()

In [19]:
# Get recommendations for the new employee
get_recommendations('emp_101', cosine_sim)

5     emp_006
15    emp_016
22    emp_023
Name: id, dtype: object